# Résolution problème Osborne 2

In [1]:
using BenchmarkTools, ForwardDiff, Distributions

In [3]:
include("../src/enlsip_functions_castor2.jl")

enlsip

In [4]:
n = 11
m = 65 
nb_eq = 0
nb_constraints = 22

22

In [6]:
# DataPoints

dataset = [1 0.0 1.366 ;
2 0.1 1.191 ;
3 0.2 1.112 ;
4 0.3 1.013 ;
5 0.4 0.991 ;
6 0.5 0.885 ;
7 0.6 0.831 ;
8 0.7 0.847 ;
9 0.8 0.786 ;
10 0.9 0.725 ;
11 1.0 0.746 ;
12 1.1 0.679 ;
13 1.2 0.608 ;
14 1.3 0.655 ;
15 1.4 0.616 ;
16 1.5 0.606 ;
17 1.6 0.602 ;
18 1.7 0.626 ;
19 1.8 0.651 ;
20 1.9 0.724 ;
21 2.0 0.649 ;
22 2.1 0.649 ;
23 2.2 0.694 ;
24 2.3 0.644 ;
25 2.4 0.624 ;
26 2.5 0.661 ;
27 2.6 0.612 ;
28 2.7 0.558 ;
29 2.8 0.533 ;
30 2.9 0.495 ;
31 3.0 0.500 ;
32 3.1 0.423 ;
33 3.2 0.395 ;
34 3.3 0.375;
35 3.4 0.538 ;
36 3.5 0.522 ;
37 3.6 0.506 ;
38 3.7 0.490 ;
39 3.8 0.478 ;
40 3.9 0.467 ;
41 4.0 0.457 ;
42 4.1 0.457 ;
43 4.2 0.457 ;
44 4.3 0.457 ;
45 4.4 0.457 ;
46 4.5 0.457 ;
47 4.6 0.457 ;
48 4.7 0.457 ;
49 4.8 0.457 ;
50 4.9 0.457 ;
51 5.0 0.457;
52 5.1 0.431 ;
53 5.2 0.431 ;
54 5.3 0.424 ;
55 5.4 0.420 ;
56 5.5 0.414 ;
57 5.6 0.411 ;
58 5.7 0.406 ;
59 5.8 0.406 ;
60 5.9 0.406 ;
61 6.0 0.406 ;
62 6.1 0.406 ;
63 6.2 0.406 ;
64 6.3 0.406 ;
65 6.4 0.406]

t = dataset[1:m,2]
y = dataset[1:m,3]

65-element Vector{Float64}:
 1.366
 1.191
 1.112
 1.013
 0.991
 0.885
 0.831
 0.847
 0.786
 0.725
 0.746
 0.679
 0.608
 ⋮
 0.424
 0.42
 0.414
 0.411
 0.406
 0.406
 0.406
 0.406
 0.406
 0.406
 0.406
 0.406

In [7]:
function r_k(x::Vector, t::Float64, y::Float64)
    rx = x[1]*exp(-x[5]*t) + x[2]*exp(-x[6]*(t-x[9])^2) + x[3]*exp(-x[7]*(t-x[10])^2) + x[4]*exp(-x[8]*(t-x[11])^2)
    return y - rx
end

function r(x::Vector)
    return [r_k(x,t[k],y[k]) for k=1:m]
end

resOsborne2 = ResidualsEval(0)

function (resOsborne2::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resOsborne2.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resOsborne2.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [18]:
((a,b) -> r_k(x0,a,b)).(t,y)

65-element Vector{Float64}:
 -0.028178923597736105
 -0.08459036635783423
 -0.055280604364534636
 -0.05552328211543123
  0.012243692056909339
 -0.012618609726489427
  0.006005055424373573
  0.08593895169392574
  0.0796850605157734
  0.06342638165978698
  0.11808969082308851
  0.0724907919714538
  0.009574370942529442
  ⋮
 -0.43695541593468396
 -0.45413098921155565
 -0.4246761699103752
 -0.3319421070808842
 -0.19275432396286107
 -0.02942353851438878
  0.12127451112610865
  0.23846972154323495
  0.31671319944342213
  0.36208780026692194
  0.3851645132282623
  0.395582126221179

In [8]:
function c(x::Vector)
    res = [x[1] - 1.31; 1.4 - x[1];
            x[2] - 0.4314 ; 0.8 - x[2];
            x[3] - 0.6336; 1.0 - x[3];
            x[4] - 0.5; 1.0 - x[4];
            x[5] - 0.5; 1.0 - x[5];
            x[6] - 0.6; 3.0 - x[6];
            x[7] - 1.0; 5.0 - x[7];
            x[8] - 4.0; 7.0 - x[8];
            x[9] - 2.0; 2.5 - x[9];
            x[10] - 4.5689; 5.0 - x[10];
            x[11] - 5.0; 6.0 - x[11]]
    return res
end
    
consOsborne2 = ConstraintsEval(0)

function (consOsborne2::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consOsborne2.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consOsborne2.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [9]:
function generate_starting_point()
    x0 = [rand(Uniform(1.31,1.4));
        rand(Uniform(0.4314 , 0.8));
        rand(Uniform(0.6336, 1.0));
        rand(Uniform(0.5, 1.0));
        rand(Uniform(0.5, 1.0));
        rand(Uniform(0.6, 3.0));
        rand(Uniform(1.0, 5.0));
        rand(Uniform(4.0, 7.0));
        rand(Uniform(2.0, 2.5));
        rand(Uniform(4.5689, 5.0));
        rand(Uniform(5.0, 6.0))]
    return x0
end

generate_starting_point (generic function with 1 method)

In [29]:
x_saved =  [1.3344098963722457
 0.5572842161127423
 0.6757364753061974
 0.8291980513226953
 0.9233565833014519
 0.9588470511477797
 1.9610314699563896
 4.055321823656234
 2.048625993866472
 4.60296578920499
 5.95212572157736]

11-element Vector{Float64}:
 1.3344098963722457
 0.5572842161127423
 0.6757364753061974
 0.8291980513226953
 0.9233565833014519
 0.9588470511477797
 1.9610314699563896
 4.055321823656234
 2.048625993866472
 4.60296578920499
 5.95212572157736

In [10]:
x0 = generate_starting_point()
e = eps(Float64)
se = sqrt(e)
enlsipOsborne2 = enlsip(x0,resOsborne2,consOsborne2,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)


****************************************
*                                      *
*          ENLSIP-JULIA-0.4.0          *
*                                      *
****************************************

Starting point : [1.3940310247118448, 0.6628673005034884, 0.6680476672895577, 0.7270399511998522, 0.8903624226702807, 1.7572700433977704, 3.9633981078763756, 5.842697373188528, 2.1873685624589507, 4.810867815379349, 5.489300014778523]

Starting point :
    1.394031e+00   6.628673e-01   6.680477e-01   7.270400e-01   8.903624e-01   1.757270e+00   3.963398e+00   5.842697e+00   2.187369e+00   4.810868e+00  
    5.489300e+00 

Number of equality constraints   : 0
Number of inequality constraints : 22
Constraints internal scaling     : false


Iteration steps information

iter     objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  3.835783e+00  0.00e+00   4.66e-01   8.412e+00   0    11   8.32e-02    0.00e+00     1.00e-01     -
  

ENLSIP(300, [1.3588184141147785, 0.5246994823641321, 0.6336, 0.5, 0.9812510883007463, 0.6731462674727923, 1.4261205777570007, 4.0, 2.34021111025807, 4.5689, 6.0], 0.4558771931598632)

In [40]:
# Comparaison x_saved avec solution du papier

x_opt = [1.31000
0.43157
0.63367
0.59941
0.75423
0.90423
1.36573
4.82393
2.39867
4.56890
5.67535]


enlsipOsborne2_x_opt = enlsip(x_opt,resOsborne2,consOsborne2,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)


enlsipOsborne2_x_saved = enlsip(x_saved,resOsborne2,consOsborne2,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)
objective_value_x_opt = enlsipOsborne2_x_opt.obj_value

@show dot(r(x_opt),r(x_opt))

@show enlsipOsborne2_x_saved.obj_value

@show enlsipOsborne2_x_opt.sol - x_opt

enlsipOsborne2.sol -  enlsipOsborne2_x_saved.sol


****************************************
*                                      *
*          ENLSIP-JULIA-0.4.0          *
*                                      *
****************************************

Starting point : [1.31, 0.43157, 0.63367, 0.59941, 0.75423, 0.90423, 1.36573, 4.82393, 2.39867, 4.5689, 5.67535]

Starting point :
    1.310000e+00   4.315700e-01   6.336700e-01   5.994100e-01   7.542300e-01   9.042300e-01   1.365730e+00   4.823930e+00   2.398670e+00   4.568900e+00  
    5.675350e+00 

Number of equality constraints   : 0
Number of inequality constraints : 22
Constraints internal scaling     : false


Iteration steps information

iter     objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  1.263083e+00  0.00e+00   6.00e-04   3.704e+00   2     9   2.70e-04    0.00e+00     1.00e-01    (1,19)
   1  1.262483e+00  0.00e+00   2.93e-01   3.814e+00   2     9   2.28e-01    8.55e-01     1.00e-01    (5,19)
   2  9.6979

11-element Vector{Float64}:
  1.5059894664659623e-8
  5.694332227967891e-9
  0.0
  0.0
  1.6945176595228872e-8
 -2.301278057004197e-9
  3.972478279656855e-8
  0.0
  3.0571506215437694e-8
  0.0
  0.0

In [36]:
@show c(x_opt)

c(enlsipOsborne2_x_saved.sol)

c(x_opt) = [0.0, 0.08999999999999986, 0.00017000000000000348, 0.36843000000000004, 6.999999999990347e-5, 0.36633000000000004, 0.09941, 0.40059, 0.25422999999999996, 0.24577000000000004, 0.30423, 2.09577, 0.3657300000000001, 3.63427, 0.8239299999999998, 2.17607, 0.3986700000000001, 0.10132999999999992, 0.0, 0.4310999999999998, 0.6753499999999999, 0.3246500000000001]


22-element Vector{Float64}:
 0.048818395937742176
 0.04118160406225768
 0.0932994753695453
 0.27530052463045473
 0.0
 0.36639999999999995
 0.0
 0.5
 0.4812510674603345
 0.018748932539665475
 0.07314627074131552
 2.3268537292586844
 0.4261205308467626
 3.5738794691532374
 0.0
 3.0
 0.3402110738629349
 0.15978892613706508
 0.0
 0.4310999999999998
 1.0
 0.0

# Résolution avec Ipopt

In [11]:
using JuMP, Ipopt

In [22]:
x_low = [1.31,0.4314,0.6336,0.5,0.5,0.6,1.0,4.0,2.0,4.5689,5.0]
x_upp = [1.4,0.8,1.0,1.0,1.0,3.0,5.0,7.0,2.5,5.0,6.0]


x_saved =  [1.3344098963722457
 0.5572842161127423
 0.6757364753061974
 0.8291980513226953
 0.9233565833014519
 0.9588470511477797
 1.9610314699563896
 4.055321823656234
 2.048625993866472
 4.60296578920499
 5.95212572157736]

model = Model(Ipopt.Optimizer)
@variable(model,x_low[i] <= x[i=1:n] <= x_upp[i], start = x_saved[i])



@NLobjective(model, Min, sum((y[i] - (x[1]*exp(-x[5]*t[i]) + x[2]*exp(-x[6]*(t[i]-x[9])^2) + x[3]*exp(-x[7]*(t[i]-x[10])^2) + 
            x[4]*exp(-x[8]*(t[i]-x[11])^2)))^2 for i=1:m))

JuMP.optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       66

Total number of variables............................:       11
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       11
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.9525163e+00 0.00e+00 4.44e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  